# Lematización para Mejora de Análisis de Sentimiento en Reseñas Multilingües
Implementar un pipeline de lematización que unifique variantes morfológicas ("corriendo" → "correr"), mejorando la precisión del modelo de clasificación.



## 1. Diagnóstico de Problemas
Objetivo: Identificar palabras no lematizadas que causan ruido en el análisis.


### 1.1 - Análisis Exploratorio
Descargar el dataset reviews_multilang.csv
Ejemplo Crítico (Español):
"Los usuarios reportaron fallas constantes: no funciona, se traba y no responde."  
Procesar sin Lematización usando CountVectorizer:
Contar frecuencias de tokens: ["funciona", "funcionar", "trabó", "responder"].
Identificar:
Variantes morfológicas que inflan el vocabulario.
Errores de POS tagging (ej: "reportaron" etiquetado como sustantivo).
Pistas:
Usar spacy.load("es_core_news_sm") para inspeccionar POS tags.
Generar una nube de palabras con WordCloud para visualizar repeticiones innecesarias.
Verificación:
Tabla con 5 pares de palabras que deben unificarse (ej: "fallas" → "falla").

### 1.2: Implementación del Lematizador
Objetivo: Crear una función que lematice texto según su idioma y categoría gramatical.

Tarea 2 - Pipeline de Lematización
Para Español:
Usar spaCy para lematizar y obtener POS tags.
Mapear verbos a infinitivo ("reportaron" → "reportar"), sustantivos a singular ("fallas" → "falla").
Para Inglés:
Usar WordNetLemmatizer de NLTK con POS tags (ej: pos='v' para verbos).
Manejar Ambiguidades:
En "El banco financiero cerró", "cerró" → "cerrar" (verbo) vs "banco" → "banco" (sustantivo).
Requisitos:
Input: "Los dispositivos fallaron constantemente, no funcionan bien."
Output Esperado: ["el", "dispositivo", "fallar", "constantemente", "no", "funcionar", "bien"].
Pista:
Filtrar stopwords después de lematizar.
Usar token.lemma_ en spaCy y lemmatizer.lemmatize(token, pos) en NLTK.
Fase 3: Optimización y Validación
Objetivo: Ajustar el lematizador para manejar jerga técnica y evaluar su impacto.


Tarea 3 - Personalización y Pruebas (Opcional)
Métricas de Rendimiento:
Entrenar un modelo de RandomForest con y sin lematización. Para predecir el sentimiento.
Comparar F1-score y tamaño del vocabulario.
Pista:
Usar PhraseMatcher de spaCy para detectar términos técnicos no lematizados.
Para "crasheó", aplicar una regla regex si el lematizador no lo resuelve.
Fase 4: Evaluación Comparativa
Objetivo: Medir la mejora en la precisión del modelo y reducir falsos negativos.


Tarea 4 - Análisis Cuantitativo
Dataset de Validación:
200 reseñas etiquetadas manualmente (50% negativas, 50% positivas).
Resultados Esperados:
Reducción de vocabulario ≥30%.
Aumento de F1-score ≥10% en reseñas con negaciones ("no funciona" vs "funcionando").
Pista:
Usar TfidfVectorizer para ponderar términos clave post-lematización.
Si el F1-score baja, revisar lemas de palabras negativas ("nunca" → "nunca").
Entrega Final
Código:
Función lematizar(texto, idioma) que maneje español e inglés.
Documentación:
Reporte PDF con:
Comparativo de métricas pre/post lematización.
Ejemplos de errores corregidos (ej: "trabó" → "trabar").

🔍 Fase 1: Diagnóstico de Problemas
Tarea 1: Análisis Exploratorio
1.1. Cargar Datos
python
import pandas as pd

df = pd.read_csv("reviews_multilang.csv")
df.head()
1.2. Procesar sin Lematización
python
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_counts = vectorizer.fit_transform(df["review_español"])  # suponiendo columna en español
token_freq = pd.DataFrame({'token': vectorizer.get_feature_names_out(), 'freq': X_counts.sum(axis=0).A1})
token_freq.sort_values(by="freq", ascending=False).head(10)
1.3. POS Tagging con spaCy
python
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

nlp = spacy.load("es_core_news_sm")
doc = nlp("Los usuarios reportaron fallas constantes: no funciona, se traba y no responde.")

for token in doc:
    print(token.text, token.pos_, token.lemma_)
1.4. Visualización: Nube de Palabras
python
text = " ".join(df["review_español"])
doc = nlp(text)
lemmas = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
wordcloud = WordCloud(background_color='white').generate(lemmas)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
1.5. Tabla de Variantes a Unificar
python
# Manualmente o con lógica heurística
variantes = [("fallas", "falla"), ("funciona", "funcionar"), ("trabó", "trabar"), ("responder", "responder"), ("reportaron", "reportar")]
pd.DataFrame(variantes, columns=["Forma Original", "Lema"])
🛠️ Fase 2: Implementación del Lematizador
Tarea 2: Pipeline de Lematización
python
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # fallback

def lematizar(texto, idioma):
    if idioma == "es":
        doc = spacy.load("es_core_news_sm")(texto)
        return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    elif idioma == "en":
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(texto)
        tagged = pos_tag(tokens)
        return [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged if word.lower() not in stopwords.words("english")]
Prueba:
python
print(lematizar("Los dispositivos fallaron constantemente, no funcionan bien.", "es"))
⚙️ Fase 3: Optimización y Validación
Tarea 3: Métricas de Rendimiento
python
# Entrenar RandomForest con y sin lematización
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocesamiento
df["review_lematizado"] = df["review_español"].apply(lambda x: " ".join(lematizar(x, "es")))

# TF-IDF
vectorizer = TfidfVectorizer()
X_lem = vectorizer.fit_transform(df["review_lematizado"])
X_raw = TfidfVectorizer().fit_transform(df["review_español"])
y = df["sentimiento"]  # asumimos que existe esta columna

# Modelo con lematización
X_train, X_test, y_train, y_test = train_test_split(X_lem, y, test_size=0.2)
clf = RandomForestClassifier().fit(X_train, y_train)
preds = clf.predict(X_test)
print("F1-score con lematización:", f1_score(y_test, preds, average="macro"))

# Sin lematización
X_train2, X_test2, _, _ = train_test_split(X_raw, y, test_size=0.2)
clf2 = RandomForestClassifier().fit(X_train2, y_train)
preds2 = clf2.predict(X_test2)
print("F1-score sin lematización:", f1_score(y_test, preds2, average="macro"))
📊 Fase 4: Evaluación Comparativa
Tarea 4: Análisis Cuantitativo
python
vocab_size_raw = len(TfidfVectorizer().fit(df["review_español"]).vocabulary_)
vocab_size_lem = len(TfidfVectorizer().fit(df["review_lematizado"]).vocabulary_)
print("Reducción de vocabulario:", round((vocab_size_raw - vocab_size_lem) / vocab_size_raw * 100, 2), "%")
📌 Entrega Final
Función Final:
python
# Ya definida como `lematizar(texto, idioma)`
Documentación Esperada:
Comparativo de métricas pre/post lematización.

Ejemplos como: "trabó" → "trabar", "funcionó" → "funcionar".

